<a href="https://colab.research.google.com/github/Nileshsri2022/genai/blob/main/Multi_Document_RAG_QA_Groq_LLAMA_3_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
#!pip install langchain-community==0.2.15 langchain-chroma==0.1.3 langchain-text-splitters==0.2.2 langchain-huggingface==0.0.3 langchain-groq==0.1.9 unstructured==0.15.0 unstructured[pdf]==0.15.0 nltk==3.8.1

In [4]:
#!apt-get install poppler-utils
#!apt install tesseract-ocr

In [6]:
import os
#text to chunks
from langchain_community.document_loaders import UnstructuredPDFLoader, DirectoryLoader
#chunks
from langchain_text_splitters import CharacterTextSplitter
from langchain_huggingface import HuggingFaceEmbeddings
#persistent directory
from langchain_chroma import Chroma
from langchain_groq import ChatGroq
from langchain.chains import RetrievalQA

In [7]:
GROQ_API_KEY = "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"

In [8]:
os.environ["GROQ_API_KEY"] = GROQ_API_KEY

In [9]:
loader = DirectoryLoader("data/", glob="./*.pdf", loader_cls=UnstructuredPDFLoader)
documents = loader.load()

In [10]:
text_splitter = CharacterTextSplitter(
    chunk_size=2000,
    chunk_overlap=500
)

text_chunks = text_splitter.split_documents(documents)

In [11]:
persist_directory = "doc_db"

In [12]:
embedding = HuggingFaceEmbeddings()

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [13]:
# load doc -> extract text -> smaller chunks -> vector embedding -> store in persistent dir
vectorstore = Chroma.from_documents(
    documents=text_chunks,
    embedding=embedding,
    persist_directory=persist_directory
)
# once the persistent dir created you only run the code after that everytime

In [14]:
retriever = vectorstore.as_retriever()

In [15]:
llm = ChatGroq(
    model="llama-3.1-8b-instant",
    temperature=0
)

In [16]:
qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=retriever,
    return_source_documents=True
)

In [17]:
query = "What does the document say about DELINEATING THE CLINICAL SYNDROME?"
response = qa_chain.invoke({"query":query})

In [18]:
print(response)

{'query': 'What does the document say about DELINEATING THE CLINICAL SYNDROME?', 'result': 'The document states that it is useful to divide Respiratory Tract Infections (RTIs) into those involving the upper and the lower tracts. \n\nThe clinical syndromes involving the upper tract include:\n\n1. Otitis media\n2. Mastoiditis\n3. Sinusitis\n4. Pharyngitis\n\nInfections of the lower tract can be divided into:\n\n1. Tracheobronchitis\n2. Bronchiolitis\n3. Pneumonia\n\nMost of these conditions can exist in acute and chronic forms. Acute disease is usually caused by viral or bacterial infections, and chronic disease is usually caused by fungi, slow-growing bacteria such as mycobacteria, bacteria adapted to persist in biofilms, and occasional less common pathogens such as parasites. Chronic infections can also develop when structural changes occur as a result of recurrent or severe acute infections, surgical intervention, or other processes that alter the structural integrity of the respirato

In [19]:
print(response["result"])

The document states that it is useful to divide Respiratory Tract Infections (RTIs) into those involving the upper and the lower tracts. 

The clinical syndromes involving the upper tract include:

1. Otitis media
2. Mastoiditis
3. Sinusitis
4. Pharyngitis

Infections of the lower tract can be divided into:

1. Tracheobronchitis
2. Bronchiolitis
3. Pneumonia

Most of these conditions can exist in acute and chronic forms. Acute disease is usually caused by viral or bacterial infections, and chronic disease is usually caused by fungi, slow-growing bacteria such as mycobacteria, bacteria adapted to persist in biofilms, and occasional less common pathogens such as parasites. Chronic infections can also develop when structural changes occur as a result of recurrent or severe acute infections, surgical intervention, or other processes that alter the structural integrity of the respiratory tract.


In [20]:
query = "What does the document say about Receptor-binding domain?"
response = qa_chain.invoke({"query":query})
print(response["result"])

The document discusses the Receptor-binding domain (RBD) of the SARS-CoV-2 spike glycoprotein in several sections. Here are the key points mentioned:

1. The RBD is a part of the S1 subunit of the spike glycoprotein and harbors the ACE2-binding site.
2. The RBD has a range of configurations, from 'up' to 'down', and only the 'up' conformation can interact with ACE2.
3. The RBD is a major target for neutralizing antibodies, and most potent neutralizing mAbs induced by vaccination or natural infection target the RBD.
4. The RBD has several conserved regions, including one that starts from the front of the left shoulder and runs down to the left flank, and another one at the bottom part of the back of the RBD.
5. Neutralizing mAbs that bind to the RBD can be grouped into several clusters, including left shoulder, neck, right shoulder, left flank, and right flank.
6. The RBD is a critical region for the interaction between the spike glycoprotein and the host cell receptor, ACE2.
7. Mutatio